#### Importing all Relevant Libraries

In [0]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

#### 1. [10 pts] What was the average time each driver spent at the pit stop for each race?

In [0]:
# Importing relevant data and converting to pandas dataframe 
# I find it easier to deal with pandas instead of spark
df_pit_stops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)
df_pit_stops = df_pit_stops.toPandas()
df_drivers = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
df_drivers = df_drivers.toPandas()
df_races = spark.read.csv('s3://columbia-gr5069-main/raw/races.csv', header=True)
df_races = df_races.toPandas()  


In [0]:
# Quick overview of pitstop dataframe
df_pit_stops.head()

In [0]:
#Running a group by function to get each driver's avg time (in miliseconds) at pit stop at each race
avg_pit_stop_time = pd.DataFrame(df_pit_stops.groupby(['raceId', 'driverId'])['milliseconds'].mean()).reset_index()

In [0]:
avg_pit_stop_time.head()

In [0]:
# Overview of driver information
df_drivers.head()

In [0]:
#Overview of Races
df_races.head()

In [0]:
#Matching on driver ID to get name of drivers on pit_stops
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_drivers[['driverId', 'forename', 'surname']], on='driverId', how='left')
#I add driver ID to Name, just to make it easier for me too visualize / see. no need to necessarily add it
avg_pit_stop_time['Name + id'] = avg_pit_stop_time['forename'] + ' ' +avg_pit_stop_time['surname'] + ' ' +avg_pit_stop_time ['driverId']
avg_pit_stop_time.drop(['forename', 'surname'], axis=1, inplace=True)
avg_pit_stop_time.head()

In [0]:
# merging on RaceId to get each unique race 
# I use race name and date of race to make sure I am getting all races
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_races[['raceId', 'name', 'date']], on='raceId', how='left')
avg_pit_stop_time['Race Name + date'] = avg_pit_stop_time['name'] + ' ' + avg_pit_stop_time['date']
avg_pit_stop_time.drop(['name','date'], axis=1, inplace=True)
avg_pit_stop_time.head()

#### 2. [20 pts] Rank the average time spent at the pit stop in order of who won each race


In [0]:
#I am getting the results dataset. 
df_results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True)
df_results = df_results.toPandas()

In [0]:
#Quick overview of player results per race
df_results.head()

In [0]:
#Merging on raceId and driver Id to get points for each player
# I use points because that way i can get all drivers, even those that did not finish the race and those that have 0 points
# I only deal with the ones not finishing the race as part of Q5.
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_results[['raceId', 'driverId', 'points']], on=['raceId', 'driverId'], how='left')
avg_pit_stop_time.head()

In [0]:
#Making sure raceId and points is categorized as int so that I can sort the values by raceiD and then points
avg_pit_stop_time.raceId = avg_pit_stop_time.raceId.astype(int)
avg_pit_stop_time.points = avg_pit_stop_time.points.astype(int)
avg_pit_stop_time = avg_pit_stop_time.sort_values(by=['raceId', 'points'], ascending=False)

In [0]:
#Now, for each race, players are ranked based on highest points to lowest points
avg_pit_stop_time

#### 3. [20 pts] Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset


In [0]:
# No codes are missing
df_drivers['code'].isnull().sum()

In [0]:
#Matching on driver ID to get code from drivers dataset
avg_pit_stop_time = pd.merge(avg_pit_stop_time, df_drivers[['driverId', 'code']], on='driverId', how='left')
avg_pit_stop_time.head()